In [1]:
from jupyter_plotly_dash import JupyterDash
import base64

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps


from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "123456"
shelter = AnimalShelter(username, password)

df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'image.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.A([
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
            height = 100, width = 100))], href = 'https://www.snhu.edu', target = "_blank"),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B('Nick Markel')),
    html.Hr(),
    
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label':'Water Rescue', 'value': 'Water'},
            {'label':'Mountain Rescue', 'value': 'Mountain'},
            {'label':'Disaster Rescue', 'value':'Disaster'},
        ],
        value='All'
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        row_selectable="single", 
        selected_rows=[],
        filter_action="native", 
        sort_action="native", 
        page_action="native",
        page_current=0, 
        page_size=10, 

    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex', 'justify-content':'center'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
]) 

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
 
    if filter_type == 'All':
        df = pd.DataFrame.from_records(shelter.read({})) 
        
    elif filter_type == 'Water':
        df = pd.DataFrame(list(shelter.read(
            { 
            "breed": {
                "$in": [
                    "German Shepherd",
                    "Alaskan Malamute",
                    "Old English Sheepdog",
                    "Siberian Husky",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        )))
        
    elif filter_type == 'Mountain':
        df = pd.DataFrame(list(shelter.read(
            { 
            "breed": {
                "$in": [
                    "German Shepherd",
                    "Alaskan Malamute",
                    "Old English Sheepdog",
                    "Siberian Husky",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        )))
        
    elif filter_type == 'Disaster':
        germanRegex = re.compile(".*german.*", re.IGNORECASE)
        goldenRegex = re.compile(".*golden.*", re.IGNORECASE)
        bloodRegex = re.compile(".*blood.*", re.IGNORECASE)
        doberRegex = re.compile(".*dober.*", re.IGNORECASE)
        rottRegex = re.compile(".*rott.*", re.IGNORECASE)
        
        df = pd.DataFrame.from_records(shelter.read({
            '$or':[
                {"breed": {'$regex': germanRegex}},
                {"breed": {'$regex': goldenRegex}},
                {"breed": {'$regex': bloodRegex}},
                {"breed": {'$regex': doberRegex}},
                {"breed": {'$regex': rottRegex}},
            ],
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
        }))
        
    return df.to_dict('records')

@app.callback(Output('datatable-id', 'style_data_conditional'), [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(Output('graph-id', "children"), [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    
    dffPie = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.pie(dffPie, names='breed',)
        )    
    ]

@app.callback(Output('map-id', "children"), [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [dl.Map(style={'width': '500px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])]


app

Connection successful
read successful
